# Dominio applicativo
il dominio prevede 4 classi: [Mazzo] [Partita] [Giocatore] [Sarsa]

In [575]:
from random import randint, random, shuffle
import numpy as np
from time import time, sleep
import pickle
import os

# Mazzo

In [576]:
class Mazzo():
    # mazzo di carte, ogni carta assume valore tra 0 a 39
    # le decine rappresentano i semi
    # le unità rappresentano i valori
    def __init__(self):
        self.carte = [i for i in range(0,40)]
        # si randomizza il mazzo
        shuffle(self.carte)

    def carteRimaste(self):
        return len(self.carte)
    
    def getUltimaCarta(self):
        return self.carte[-1]

    def pesca(self):
        return self.carte.pop(0)

    def reset(self):
        self.__init__()

# Sarsa

stato =	(puntiAvversarioAlmeno45, puntiCartaInFondoAlmeno10, briscola, fasciaBriscoleUscite,
	almenoUncaricoDenaraUscito, almenoUnCaricoSpadeUscito, almenoUnCaricoBastoniUscito, almenoUnCaricoCoppeUscito, semeCartaAvversario, fasciaPuntiCartaAvversario, semeCarta0, fasciaPuntiCarta0, semeCarta1, fasciaPuntiCarta1, semeCarta2, fasciaPuntiCarta2)

azione = 0, 1, 2

In [577]:
class Sarsa():

    def __init__(self):
        # dizionario tupla (azione + stato) -> valore
        self.Q = {}
        self.ultimoStato = None
        self.ultimaAzione = -1
    
    def eps_greedy(self, mano, s, eps=0.1):
        if np.random.uniform(0,1) < eps:
            # scelgo una azione randomica
            self.ultimoStato = s
            self.ultimaAzione = randint(0,len(mano)-1)
            return self.ultimaAzione
        else:
            # scelgo l'azione in base alla greedy policy
            return self.greedy(mano, s)
        

    # restituisco l'indice dell'azione (0, 1, 2) corrispondente al massimo valore della coppia azione-stato
    def greedy(self, mano, s):
        self.ultimoStato = s
        # non c'è nulla da scegliere
        if len(mano) == 1: 
            self.ultimaAzione = 0
            return self.ultimaAzione
        
        valori = []
        for a in range(len(mano)):
            valori.append(self.cercaValore(s,a))
        
        # se nessuna azione è stata precedentemente esplorata 
        # seleziono una azione casuale
        if sum(valori) == 0:
            self.ultimaAzione = randint(0,len(mano)-1)
            return self.ultimaAzione
        
        valoreMassimo = max(valori)
        self.ultimaAzione = valori.index(valoreMassimo)
        return self.ultimaAzione
    
    # restituisce il valore appreso in merito alla coppia stato-azione
    def cercaValore(self, s, a):
        if self.Q.get((a,)+s) == None: return 0
        return self.Q.get((a,)+s)
    
    def aggiornaPolitica(self, mano, alpha, gamma, rew, stato_successivo):
        azioneSuccessiva = self.eps_greedy(mano, stato_successivo)
        valorePrecedente = self.Q.get((self.ultimaAzione,) + self.ultimoStato)
        valoreSuccessivo = self.Q.get((azioneSuccessiva,) + stato_successivo)

        if(valorePrecedente == None): valorePrecedente = 0
        if(valoreSuccessivo == None): valoreSuccessivo = 0

        aggiornamento = valorePrecedente + alpha * (rew + gamma * valoreSuccessivo - valorePrecedente)
        self.Q[(self.ultimaAzione,) + self.ultimoStato] = aggiornamento
        
        # return azioneSuccessiva

        
    

# Giocatore

In [578]:
class Giocatore():

    def __init__(self, tipo = 0):
        self.mano = []
        self.punti = 0
        # tipo indica la modalità con cui viene scleta la mossa del giocatore
        # tipo 0 -> giocatore post-apprendimento
        # tipo 1 -> giocaore apprendimento 
        # tipo 2 -> giocatore casuale
        self.tipo = tipo
        self.ia = Sarsa()
    
    def reset(self):
        self.mano = []
        self.punti = 0
        

    def pesca(self, mazzo):
        self.mano.append(mazzo.pesca())
    
    # restituisce un intero tra quelli presenti nella mano
    def gioca(self, s):
        match self.tipo:
            case 0: return self.mossaOttimale(s)
            case 1: return self.apprendi(s)
            case 2: return self.mossaCasuale()
        
    def mossaOttimale(self, s):
        # gestione della mossa scelta tramite la policy ottima
        return  self.mano.pop(self.ia.greedy(self.mano, s))

    def apprendi(self, s):
        # gestione apprendimento
        return self.mano.pop(self.ia.eps_greedy(self.mano, s))
    
    def mossaCasuale(self):
        n = randint(0, 2)
        return self.mano.pop(n)

    def setPunti(self, punti):
        self.punti += punti
    
    def getPunti(self):
        return self.punti
    
    def getMano(self):
        return self.mano

    def setReward(self, mano, alpha, gamma, rew, stato_successivo):
        return self.ia.aggiornaPolitica(mano, alpha, gamma, rew, stato_successivo)
    
    def cambiaTipo(self, tipo):
        self.tipo = tipo

# Environment

In [579]:
class Environment():
    
    WIN_REWARD = 200
    SOGLIA_VITTORIA = 45
    BRISCOLA_ALTA = 10
    SOGLIA_BRISCOLE = 7

    # tipo indica il tipo dei giocatori che giocano la partita
    # tipo 0 -> giocatore post-apprendimento
    # tipo 1 -> giocaore apprendimento 
    # tipo 2 -> giocatore casuale
    def __init__(self, tipo1 = 0, tipo2 = 0):
        self.giocatori = (Giocatore(tipo1), Giocatore(tipo2))
        self.vittorie = (0,0) # giocatore 1 giocatore2
        self.mazzo = Mazzo()
        # info sullo stato di una partita
        self.briscola = self.mazzo.getUltimaCarta()
        self.diTurno = 0
        self.uscitoCaricoSeme = [False, False, False, False] #denara spade bastoni coppe
        self.briscoleUscite = 0 
    
    def reset(self):
        self.giocatori[0].reset()
        self.giocatori[1].reset()
        self.mazzo.reset()
        # alterno il giocatore che inizia
        self.diTurno = (self.diTurno + 1) % 2 

        # ripristino le statistiche sulla partita
        self.briscola = self.mazzo.getUltimaCarta()
        self.uscitoCaricoSeme = [False, False, False, False] 
        self.briscoleUscite = 0 
    
    def episodio(self, alpha, gamma):
        # i giocatori pescano 3 carte
        for _ in range(0, 3):
            self.giocatori[self.diTurno].pesca(self.mazzo)
            self.giocatori[(self.diTurno + 1) % 2].pesca(self.mazzo)

        partitaFinita = False
        while(self.mazzo.carteRimaste() > 0 and not partitaFinita):
            partitaFinita = self.step(alpha, gamma)

        self.reset()
    
    def addestramento(self, alpha=0.1, num_episodes=10000, eps=0.3, gamma=0.95, eps_decay=0.00005):
        for _ in range(num_episodes):
            # decremento il valore di epsilon al crescere degli episodi
            if eps > 0.01:
                eps -= eps_decay

            self.episodio(alpha, gamma)
        
        # a scopo di debug
        print(self.vittorie) 

        self.vittorie = (0,0)
        self.giocatori[0].cambiaTipo(0)
        self.giocatori[1].cambiaTipo(2)

        for _ in range(10000):
            self.episodio(alpha, gamma)
        print(self.vittorie) 
        
        #print(self.giocatori[0].ia.Q)
    
    # restituisce True se un giocatore ha vinto, False altrimenti
    def step(self, alpha, gamma):
        # genero lo stato attuale della partita
        avversarioAllaSogliaVittoria = self.giocatori[(self.diTurno + 1) % 2].getPunti() > self.SOGLIA_VITTORIA
        briscolaAlta = self.calcolaPunti(self.briscola) >= self.BRISCOLA_ALTA
        if self.briscoleUscite < self.SOGLIA_BRISCOLE: fasciaBriscole = 0
        else: fasciaBriscole = self.briscoleUscite
        s = (avversarioAllaSogliaVittoria, briscolaAlta, self.briscola // 10, 
                fasciaBriscole) + tuple(self.uscitoCaricoSeme) + self.statoMano(self.giocatori[self.diTurno].getMano())
        
        # viene effettuata la prima giocata
        primaGiocata = self.giocatori[self.diTurno].gioca(s)

        # genero lo stato della partita aggiornato alla prima giocata
        avversarioAllaSogliaVittoria = self.giocatori[self.diTurno].getPunti() > self.SOGLIA_VITTORIA
        semePrimaGiocata = primaGiocata // 10
        valorePrimaGiocata = self.calcolaPunti(primaGiocata)
        s = (avversarioAllaSogliaVittoria, briscolaAlta, self.briscola // 10,  
                fasciaBriscole) + tuple(self.uscitoCaricoSeme) + (semePrimaGiocata, 
                    valorePrimaGiocata) + self.statoMano(self.giocatori[(self.diTurno + 1) % 2].getMano())
        
        # viene effettuata la seconda giocata
        secondaGiocata = self.giocatori[(self.diTurno + 1) % 2].gioca(s)

        # valutazione = (vincitore, punti)
        giocate = (primaGiocata, secondaGiocata)
        valutazione = self.valutaGiocate(giocate)

        # assegno i punti al giocatore che vince la mano
        self.giocatori[valutazione[0]].setPunti(valutazione[1])

        # ridistribuisco le carte
        self.giocatori[self.diTurno].pesca(self.mazzo)
        self.giocatori[(self.diTurno + 1) % 2].pesca(self.mazzo)  

        # genero gli stati successivi della partita
        for giocata in giocate:
            if self.calcolaPunti(giocata) >= self.BRISCOLA_ALTA:
                self.uscitoCaricoSeme[giocata // 10] = True
            if (giocata // 10) == (self.briscola // 10):
                self.briscoleUscite += 1
        
        if self.briscoleUscite < self.SOGLIA_BRISCOLE: fasciaBriscole = 0
        else: fasciaBriscole = self.briscoleUscite
        
        # stato aggiornato del giocatore di turno
        avversarioAllaSogliaVittoria = self.giocatori[(self.diTurno + 1) % 2].getPunti() > self.SOGLIA_VITTORIA
        s0 = (avversarioAllaSogliaVittoria, briscolaAlta, self.briscola // 10,
               fasciaBriscole) + tuple(self.uscitoCaricoSeme) + self.statoMano(self.giocatori[self.diTurno].getMano())
        
        # stato aggiornato del giocatore non di turno
        avversarioAllaSogliaVittoria = self.giocatori[self.diTurno].getPunti() > self.SOGLIA_VITTORIA
        s1 = (avversarioAllaSogliaVittoria, briscolaAlta, self.briscola // 10, 
              fasciaBriscole) + tuple(self.uscitoCaricoSeme) + self.statoMano(self.giocatori[self.diTurno].getMano())

        # controllo la vittoria e assegno i reward (manca l'inserimento della giocata avversaria nello stato)
        if(self.giocatori[self.diTurno].getPunti() > 60): 
            self.vittorie = (self.vittorie[self.diTurno]+1, self.vittorie[(self.diTurno + 1) % 2])
            self.giocatori[self.diTurno].setReward(self.giocatori[self.diTurno].getMano(), alpha, gamma, self.WIN_REWARD, s0)
            self.giocatori[(self.diTurno + 1) % 2].setReward(self.giocatori[(self.diTurno + 1) % 2].getMano(), alpha, gamma, -self.WIN_REWARD, s1)
            return True
        
        if(self.giocatori[(self.diTurno + 1) % 2].getPunti() > 60): 
            self.vittorie = (self.vittorie[self.diTurno]+1, self.vittorie[(self.diTurno + 1) % 2])
            self.giocatori[self.diTurno].setReward(self.giocatori[self.diTurno].getMano(), alpha, gamma, -self.WIN_REWARD, s0)
            self.giocatori[(self.diTurno + 1) % 2].setReward(self.giocatori[(self.diTurno + 1) % 2].getMano(), alpha, gamma, self.WIN_REWARD, s1)
            return True
        
        if(self.giocatori[self.diTurno].getPunti() == 60 and self.giocatori[(self.diTurno + 1) % 2].getPunti() == 60):
            self.giocatori[self.diTurno].setReward(self.giocatori[self.diTurno].getMano(), alpha, gamma, 0, s0)
            self.giocatori[(self.diTurno + 1) % 2].setReward(self.giocatori[(self.diTurno + 1) % 2].getMano(), alpha, gamma, 0, s1)

        if(valutazione[0] == self.diTurno):
            self.giocatori[self.diTurno].setReward(self.giocatori[self.diTurno].getMano(), alpha, gamma, valutazione[1], s0)
            self.giocatori[(self.diTurno + 1) % 2].setReward(self.giocatori[(self.diTurno + 1) % 2].getMano(), alpha, gamma, -valutazione[1], s1)   

        self.giocatori[self.diTurno].setReward(self.giocatori[self.diTurno].getMano(), alpha, gamma, -valutazione[1], s0)
        self.giocatori[(self.diTurno + 1) % 2].setReward(self.giocatori[(self.diTurno + 1) % 2].getMano(), alpha, gamma, valutazione[1], s1)   
        
        # aggiorno il giocatore di turno
        self.diTurno = valutazione[0] 

        return False
    
    def statoMano(self, mano):
        s = []
        for carta in mano:
            s.append(carta // 10)
            s.append(self.calcolaPunti(carta))
        return tuple(s)

    # restituisce la tupla (vincitore, punti)   
    def valutaGiocate(self, giocate):
        semi = (giocate[0]//10, giocate[1]//10)
        punti = (self.calcolaPunti(giocate[0]), self.calcolaPunti(giocate[1]))

        if semi[0] == semi[1]:
            # 0 -> giocatore di turno
            if giocate[0] > giocate[1]: return (self.diTurno, punti[0] + punti[1])
            return ((self.diTurno + 1) % 2, punti[0] + punti[1])

        if semi[1] == self.briscola//10: return ((self.diTurno + 1) % 2, punti[0] + punti[1])
        return (self.diTurno, punti[0] + punti[1])

    def calcolaPunti(self, carta):
        valore = carta%10 + 1
        if valore == 1: return 11
        if valore == 3: return 10
        if valore < 8: return 0
        return valore - 6

In [580]:
gioco = Environment(1, 1)
gioco.addestramento(0.1,3000000)

KeyboardInterrupt: 